# Modelos LSTM

## 1. Introducción a LSTM

LSTM es un acrónimo de *Long-Short Term Memory* y representa a un **subtipo de RNN** (*Recurrent Neural Network*) capaz de **retener información relevante** sobre datos ya procesados que ayude al procesamiento de nuevas secuencias de datos completas. Su arquitectura se encuentra compuesta a su vez por tres redes neuronales:

* ***Forget Gate***: este primer modelo es el encargado de filtrar qué información previa es útil para su almacenamiento y qué datos ya no son útiles para futuras iteraciones. 

* ***Input Gate***: esta segunda red trata de determinar el valor que presentan los datos entrantes para resolver la tarea de clasificación.

* ***Output Gate***: finalmente esta red calcula las salidas del modelo LSTM que dependerán de la tarea de clasificación que se pretende abordar.

### 1.1. Condiciones de uso

Dependiendo del framework que se pretenda utilizar (Tensorflow, Keras, Pytorch) existen diferentes tratamientos de datos y requisitos de implementación que se deben cumplir al definir la arquitectura, entrenamiento y validación de modelos. En mi caso particular he optado por utilizar **Keras** debido a la experiencia previa que tengo con la librería y a su facilidad de uso. 

1. **Procesamiento y limpieza** de los documentos.

2. **Tokenización** de los documentos especificando un token para aquellos términos que no sean reconocidos dentro de un vocabulario de palabras.

3. **Codificación** numérica en forma de matrices secuenciales de valores. 

5. **Normalización** de las secuencias numéricas para establecer un mismo tamaño fijo, completando con ceros aquellas de menor longitud y separando en varias secuencias aquellas que dispongan de un mayor tamaño.

6. Definición de la arquitectura de un **modelo** e instanciación para su posterior entrenamiento y validación.

### 1.2. Casos de uso

* Detección y extracción de patrones en secuencias de datos.
* Modelado del lenguaje natural.
* Traducción de texto.
* Reconocimiento de textos manuscritos.
* Generación de imágenes mediante mecanismos de atención.
* Sistemas de preguntas y respuestas.
* Conversión de vídeo a texto.

## 2. Estructura del notebook

1. Introducción a LSTM
2. Estructura del notebook
3. Instalación y carga de librerías
4. Lectura y carga de datos
5. Experimentos y modelos
6. Conclusiones

## 3. Instalación y carga de librerías

Este apartado tiene como único propósito cargar las librerías y dependencias necesarias para la ejecución de este notebook, así como las funciones propiamente desarrolladas. Previo a ello deberán ser instaladas bien ejecutando el script *setup.sh* mediante el comando `bash setup.sh` con permisos de ejecución en distribuciones Linux, o bien ejecutando el compando `pip install -r requirements.txt`.

In [127]:
%%capture
import sys
sys.path.append("../scripts")

# Import data read and compute functions
from data import read_train_dataset, read_test_dataset

# Import text preprocess functions
from processing import *

# numpy: to work with numeric codifications and embeddings
import numpy as np

# keras: to define and build LSTM models
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import LSTM, Activation, Dense, Input, Embedding
from keras.models import Model

# sklearn: to plot a confusion matrix per trained model
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

# matplotlib: to plot charts
import matplotlib.pyplot as plt

## 4. Lectura y carga de datos originales

En esta sección se pretende **cargar los datasets de entrenamiento y validación** procedentes de los correspondientes ficheros situados en la carpeta *data*. Al tener un **formato TSV** se deben leer como tablas aunque posteriormente se trabaje con ellos en formato *dataframe*. 

Tal y como se puede comprobar en los siguientes resultados las dimensiones de sendos conjuntos de datos se detallan a continuación:

* Conjunto de entrenamiento: **6.977 muestras**.
* Conjunto de validación: **4.368 muestras**.

In [128]:
# Read EXIST datasets
train_df = read_train_dataset()
test_df = read_test_dataset()

# Show the dimensions of the datasets
print("Train dataset dimensions:", train_df.shape)
print("Test dataset dimensions:", test_df.shape)

Train dataset dimensions: (6977, 7)
Test dataset dimensions: (4368, 7)


## 5. Experimentos y modelos

Esta sección pretende detallar los experimentos que se realizan a través de la combinación de diferentes técnicas de procesamiento de textos, codificación de documentos y arquitecturas de modelos LSTM. Como se trata de **experimentos no determinísticos**, es decir, los resultados difieren en varias ejecuciones aún con la misma configuración, la estrategia a seguir consiste en realizar **30 iteraciones de cada experimento** para luego calcular la **media de accuracy y AUC**, las métricas de evaluación escogidas para medir la calidad de un clasificador. 

Por lo tanto las siguientes secciones contienen los detalles del conjunto de experimentos realizados y las conclusiones comparativas alcanzadas, incluyendo el código, la configuración y los resultados únicamente del experimento con mejor rendimiento con respecto a las métricas de evaluación mencionadas.

Previo al comienzo de la experimentación se definen tres funciones comunes para el tratamiento y codificación de documentos, carga de embeddings pre-entrenados y validación de modelos.

In [129]:
def get_train_test_matrix(
    max_n_words: int, sequence_len: int, 
    lemm: bool = False, stemm: bool = False):
    """
    Process the train and test documents to then convert them
    into numeric sequence matrixes so the datasets can be
    used to train a LSTM model.

    Parameters
    ----------
    max_n_words : int
        Maximum number of words to keep within the LSTM memory
        based on computing the word frequency.
    sequence_len : int
        Maximum lenght of all sequences.
    lemm : bool (optional)
        True to apply lemmatization to the train and test documents.
    stemm : bool (optional)
        True to apply stemming to the train and test documents.

    Returns
    -------
    A dictionary with the following keys:
        - 'tokenizer': a Keras Tokenizer object based on the train documents
        that contains the vocabulary to then be used to create the embeddings.
        - 'train_matrix', 'test_matrix': the numeric sequence matrixes
        after converting the train and test documents.
        - 'train_labels', 'test_labels': two numeric lists which contains
        the encoded class labels for train and test datasets.
    """
    # Process train and test text documents
    processed_df = process_encode_datasets(
        train_df=train_df, 
        test_df=test_df,
        lemm=lemm, 
        stemm=stemm
    )

    # Processed train texts and encoded train labels 
    train_texts = list(processed_df["train_df"]["cleaned_text"].values)
    train_labels = processed_df["encoded_train_labels"]

    # Processed test texts and encoded test labels
    test_texts = list(processed_df["test_df"]["cleaned_text"].values)
    test_labels = processed_df["encoded_test_labels"]

    # Createa a tokenizer based on train texts
    tokenizer = Tokenizer(num_words=max_n_words)
    tokenizer.fit_on_texts(train_texts)

    # Transform each text into a numeric sequence
    train_sequences = tokenizer.texts_to_sequences(train_texts)

    # Transform each numeric sequence into a 2D vector
    train_matrix = pad_sequences(
        sequences=train_sequences, 
        maxlen=sequence_len)

    # Tokenize the test documents using the prior trained tokenizer
    test_sequences = tokenizer.texts_to_sequences(test_texts)

    # Transform each numeric sequence into a 2D vector
    test_matrix = pad_sequences(
        sequences=test_sequences,
        maxlen=sequence_len)

    return {
        "tokenizer": tokenizer,
        "train_matrix": train_matrix,
        "train_labels": train_labels,
        "test_matrix": test_matrix,
        "test_labels": test_labels
    }

In [130]:
def get_embedding_matrix(embedding_file: str, tokenizer: Tokenizer, sequence_len: int):
    """
    Load the embeddings stored in the provided file to then
    create a matrix with the numeric encoding of each
    available word within the tokenizer vocabulary.

    Parameters
    ----------
    embedding_file : str
        The path to the file which contains a set of embeddings
    tokenizer : Tokenizer (Keras)
        A trained Keras tokenizer which contains the vocabulary
        of the documents to use during the training of models
    sequence_len : int
        Maximum lenght of all embeddings.

    Returns
    -------
    A Numpy ndarray which represents an embedding matrix.
    """
    # Load the embeddings stored in a TXT file
    embedding_file = open(embedding_file)

    # Store each word with its embeddings
    embeddings_index = {
        line.split()[0]:np.asarray(line.split()[1:], dtype="float32") 
        for line in embedding_file
    }

    # Initialize the embedding matrix with zeros
    embedding_matrix = np.zeros(shape=(len(tokenizer.word_index)+1, sequence_len))

    # Complete the matrix with the prior loaded embeddings
    for word, i in tokenizer.word_index.items():
        # Search for the embeddings of each word
        embedding_vector = embeddings_index.get(word)

        # Words not found will be zeros
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix    

In [131]:
def validate_lstm_model(
    model: Model, 
    train_matrix: np.ndarray, train_labels: list, 
    test_matrix: np.ndarray, test_labels: list,
    metrics_filename: str = None, conf_matrix_filename: str = None):
    """
    Evaluates the provided trained LSTM model over the 
    train and test datasets to get the accuracy, AUC and
    a confusion matrix. To create the predictions for a
    binary classification a threshold has been set:
        - <= 0.5 represents the negative class (non-sexist).
        - > 0.5 represents the positive class (sexist).

    Parameters
    ----------
    model : Keras model
        A trained Keras model to be evaluated.
    train_matrix : Numpy ndarray
        A numeric sequence matrix with the trained documents.
    train_labels : list
        A numeric list with the class labels of the train dataset.
    test_matrix : Numpy ndarray
        A numeric sequence matrix with the test documents.
    test_labels : list
        A numeric list with the class labels of the test dataset.
    metrics_filename : str (optional)
        A path and filename to save the metrics over the 
        train and test datasets in a TXT file.
    conf_matrix_filename : str (optional)
        A path and filename to save the confusion matrix in
        a PNG image.

    Returns
    -------
    None.
    """
    # Compute and print the accuracy and AUC over train
    train_acc = model.evaluate(
        x=train_matrix, 
        y=np.array(train_labels))

    print(f"Accuracy over train dataset: {train_acc[1]}")
    print(f"AUC over train dataset: {train_acc[2]}\n")

    # Compute and print the accuracy and AUC over test
    test_acc = model.evaluate(
        x=test_matrix, 
        y=np.array(test_labels))

    print(f"Accuracy over test dataset: {test_acc[1]}")
    print(f"AUC over test dataset: {test_acc[2]}")

    # Generate class label predictions over the test dataset
    # Class 0 ~ <= 0.5 | Class 1 ~ > 0.5
    test_preds = (model.predict(test_matrix) > 0.5).astype("int32")

    # Plot the confusion matrix 
    ConfusionMatrixDisplay(
        confusion_matrix=confusion_matrix(
            np.array(test_labels), 
            np.array(test_preds)), 
        display_labels=["Non-sexist", "Sexist"]) \
    .plot()    

    # Save the confusion matrix in an image
    if (type(conf_matrix_filename) == str and
        len(conf_matrix_filename) > 0):
        plt.savefig(conf_matrix_filename)

    # Save the metrics in a text file
    if (type(metrics_filename) == str and
        len(metrics_filename) > 0):
        opened_file = open(metrics_filename, "w")
        print(f"Accuracy over train dataset: {train_acc[1]}", file=opened_file) 
        print(f"AUC over train dataset: {train_acc[2]}\n", file=opened_file) 
        print(f"Accuracy over test dataset: {test_acc[1]}", file=opened_file) 
        print(f"AUC over test dataset: {test_acc[2]}", file=opened_file) 